In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
stop = stopwords.words('english')
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm.keras import TqdmCallback
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vipin-16319/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('/Users/vipin-16319/Downloads/train-sample.csv')
df = df.drop(columns=['PostId','PostCreationDate','OwnerUserId','OwnerCreationDate','ReputationAtPostCreation','OwnerUndeletedAnswerCountAtPostTime','PostClosedDate'],axis=1)
df.shape

(140272, 8)

In [3]:
df['OpenStatus']= df['OpenStatus'].map({'open': 0, 'not a real question': 1, 'off topic': 2, 'not constructive': 3, 'too localized': 4}) 
df.head()

,Title,BodyMarkdown,Tag1,Tag2,Tag3,Tag4,Tag5,OpenStatus
0,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,mongodb,NaN,NaN,NaN,NaN,0
1,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,dom,xsd,jaxp,NaN,NaN,0
2,Too many lookup tables,What are the adverse effects of having too man...,sql-server,database-design,enums,NaN,NaN,0
3,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,php,vb.net,NaN,NaN,NaN,4
4,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",mongodb,spring-data,NaN,NaN,NaN,0


In [4]:
df['Tag1']=df['Tag1'].replace(np.nan,' ')
df['Tag2']=df['Tag2'].replace(np.nan,' ')
df['Tag3']=df['Tag3'].replace(np.nan,' ')
df['Tag4']=df['Tag4'].replace(np.nan,' ')
df['Tag5']=df['Tag5'].replace(np.nan,' ')
df['Tags']=df['Tag1']+' '+df['Tag2']+' '+df['Tag3']+' '+df['Tag4']+' '+df['Tag5']

In [5]:
df=df.drop(['Tag1','Tag2','Tag3','Tag4','Tag5'],axis=1)
df.head()

,Title,BodyMarkdown,OpenStatus,Tags
0,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,0,mongodb
1,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,0,dom xsd jaxp
2,Too many lookup tables,What are the adverse effects of having too man...,0,sql-server database-design enums
3,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,4,php vb.net
4,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",0,mongodb spring-data


In [6]:
df['combined_data'] = df['Title'] + ' ' + df['BodyMarkdown'] + ' ' + df['Tags']
df = df.drop(columns=['Title','BodyMarkdown','Tags'],axis=1)
df.head()

,OpenStatus,combined_data
0,0,For Mongodb is it better to reference an objec...
1,0,How to insert schemalocation in a xml document...
2,0,Too many lookup tables What are the adverse e...
3,4,What is this PHP code in VB.net I am looking f...
4,0,Spring-Data mongodb querying multiple classes ...


In [7]:
def preprocess_text(text):
    text = text.lower()
    unwanted_chars = [
        ",",
        "+",
        ":",
        "?",
        "!",
        '"',
        "(",
        ")",
        "'",
        ".",
        "%",
        "[",
        "]",
        "/",
        "#",
        "-",
        "@",
        "$",
        "*",
        "_",
    ]
    clean_text = " "
    for char in text:
        if char not in unwanted_chars:
            clean_text += char

    words = clean_text.split()
    filter_text = [word for word in words if len(word) > 1 or word.isdigit()]
    text = " ".join(filter_text)
    return text

In [8]:
df['cleaned_data'] = df['combined_data'].apply(preprocess_text) 

In [9]:
df = df.drop(columns=['combined_data'],axis=1)

In [10]:
df

,OpenStatus,cleaned_data
0,0,for mongodb is it better to reference an objec...
1,0,how to insert schemalocation in xml document v...
2,0,too many lookup tables what are the adverse ef...
3,4,what is this php code in vbnet am looking for ...
4,0,springdata mongodb querying multiple classes s...
...,...,...
140267,0,is it possible to implement bitwise operators ...
140268,0,ruby on rails mysql gem does not work uninitia...
140269,1,deleting image from image folder am working wi...
140270,1,need help making htmls hi to all the gurus out...


In [11]:
df['cleaned_data'] = df['cleaned_data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [12]:
X = df['cleaned_data']
y = df['OpenStatus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112217,), (28055,), (112217,), (28055,))

In [14]:
"""
****************Start of word2vec and sequential layer testing :)*********************
"""
train_sentences = X_train.tolist()
test_sentences = X_test.tolist()
train_sentences = [word_tokenize(sentence) for sentence in train_sentences]
test_sentences = [word_tokenize(sentence) for sentence in test_sentences]

model_w2v = Word2Vec(sentences=train_sentences, vector_size=100, window=5, min_count=1, workers=4)

def sentence_to_vector(sentence, model_w2v):
    vector_list = [model_w2v.wv[word] for word in sentence if word in model_w2v.wv.key_to_index]
    if len(vector_list) == 0:
        return np.zeros(model_w2v.vector_size)
    return np.mean(vector_list, axis=0)

train_vectors = np.array([sentence_to_vector(sentence, model_w2v) for sentence in train_sentences])
test_vectors = np.array([sentence_to_vector(sentence, model_w2v) for sentence in test_sentences])


In [15]:
train_vectors_norm = train_vectors / np.linalg.norm(train_vectors, axis=1, keepdims=True)
test_vectors_norm = test_vectors / np.linalg.norm(test_vectors, axis=1, keepdims=True)

num_classes = 5
y_train_encoded = to_categorical(y_train, num_classes=num_classes)
y_test_encoded = to_categorical(y_test, num_classes=num_classes)

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=model_w2v.vector_size))
model.add(Dense(64, activation='relu',  kernel_initializer='he_normal'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_vectors_norm, y_train_encoded, validation_data=(test_vectors_norm, y_test_encoded), epochs=10, verbose=0, callbacks=[TqdmCallback(verbose=1)])

y_pred = model.predict(test_vectors_norm)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
"""
*************************END OF WORD2VEC AND SEQUENTIAL :( 👎🏻 ***************************
"""


Metal device set to: Apple M1 Pro


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-05-27 23:32:05.943862: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-27 23:32:06.434263: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-27 23:32:06.438293: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-27 23:32:39.585424: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-27 23:32:39.587556: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto


 93/877 [==>...........................] - ETA: 1s

2023-05-27 23:38:16.074726: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-27 23:38:16.075760: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto


877/877 [==============================] - 1s 2ms/step
Accuracy: 0.6446266262698271
Precision: 0.6209686798547602
Recall: 0.6446266262698271
F1 Score: 0.6205620276590978


'\n*************************END OF WORD2VEC AND SEQUENTIAL :( 👎🏻 ***************************\n'

In [19]:
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

train_embeddings = model.encode(train_sentences)
test_embeddings = model.encode(test_sentences)

train_embeddings_norm = normalize(train_embeddings, norm='l2')
test_embeddings_norm = normalize(test_embeddings, norm='l2')


In [33]:
"""
********************START OF CNN WITH all-miniLM-L6-v2 EMBEDDINGS******************
"""
input_dim = 1 
input_length = train_embeddings_norm.shape[1]  

train_data = np.reshape(train_embeddings_norm, (train_embeddings_norm.shape[0], input_length, input_dim))
test_data = np.reshape(test_embeddings_norm, (test_embeddings_norm.shape[0], input_length, input_dim))

def build_model(filters=64, kernel_size=3, pool_size=2, dense_units=64):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(input_length, input_dim)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=build_model, verbose=0)

history = model.fit(train_data, y_train_encoded, validation_data=(test_data, y_test_encoded), epochs=10, verbose=0, callbacks=[TqdmCallback(verbose=1)])

y_pred = model.predict(test_data)

y_pred_classes = np.argmax(y_pred)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")




"""
************************END OF CNN WITH all-miniLM-L6-v2 MODEL************************
"""


/var/folders/y6/c7mpbpm93zq8g372zm98wyxxrshyks/T/ipykernel_1536/2527682344.py:20: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model, verbose=0)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-05-28 00:18:01.719930: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:18:01.724843: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:18:38.002697: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:18:38.005248: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto


 85/877 [=>............................] - ETA: 1s

2023-05-28 00:25:10.612934: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:25:10.614354: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto


877/877 [==============================] - 2s 2ms/step
Accuracy: 0.5431117447870255
Precision: 0.4965263235860988
Recall: 0.5431117447870255
F1 Score: 0.4886309475185096


'\n************************END OF CNN WITH all-miniLM-L6-v2 MODEL************************\n'

In [38]:
"""
********************START OF LSTM WITH all-miniLM-L6-v2 EMBEDDINGS******************
"""
input_dim = 1 
input_length = train_embeddings_norm.shape[1]  
from tensorflow.keras.layers import LSTM

train_data = np.reshape(train_embeddings_norm, (train_embeddings_norm.shape[0], input_length, input_dim))
test_data = np.reshape(test_embeddings_norm, (test_embeddings_norm.shape[0], input_length, input_dim))

def build_model(lstm_units=128, dense_units=128):
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(input_length, input_dim)))
    model.add(Dense(dense_units, activation='relu', kernel_initializer='he_normal'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=build_model, verbose=0)

history = model.fit(train_data, y_train_encoded, validation_data=(test_data, y_test_encoded), epochs=10, verbose=0, callbacks=[TqdmCallback(verbose=1)])

y_pred = model.predict(test_data)

y_pred_classes = np.argmax(y_pred)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

"""
************************END OF LSTM WITH all-miniLM-L6-v2 MODEL************************
"""


/var/folders/y6/c7mpbpm93zq8g372zm98wyxxrshyks/T/ipykernel_1536/2700133248.py:19: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model, verbose=0)


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

2023-05-28 00:37:04.563493: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:37:04.584219: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:37:04.623300: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:37:04.625602: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:37:05.035753: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 00:37:05.038964: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer fa

  9/877 [..............................] - ETA: 12s

2023-05-28 01:04:06.097727: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 01:04:06.105338: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 01:04:06.120301: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto
2023-05-28 01:04:06.121904: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Unparseable tensorflow.GraphDef proto


877/877 [==============================] - 11s 13ms/step
Accuracy: 0.5134200677241133
Precision: 0.4300575938593224
Recall: 0.5134200677241133
F1 Score: 0.3947287024935175


/Users/vipin-16319/miniforge3/envs/tensorflow_crmpy/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


'\n************************END OF LSTM WITH all-miniLM-L6-v2 MODEL************************\n'

In [46]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

mlp = MLPClassifier(hidden_layer_sizes=(150,), max_iter=300, alpha=1e-4,
                    solver='sgd', verbose=0, random_state=1,
                    learning_rate_init=.1)

for i in tqdm(range(100), dynamic_ncols=True):
    mlp.partial_fit(train_embeddings_norm, y_train_encoded, classes=np.unique(y_train_encoded))

y_pred_encoded = mlp.predict(test_embeddings_norm)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

print(classification_report(y_test, y_pred))




  0%|                                                                                                                                                                                         | 0/100 [00:00<?, ?it/s]

  1%|█▊                                                                                                                                                                               | 1/100 [00:00<01:25,  1.16it/s]

  2%|███▌                                                                                                                                                                             | 2/100 [00:01<01:30,  1.08it/s]

  3%|█████▎                                                                                                                                                                           | 3/100 [00:02<01:24,  1.15it/s]

  4%|███████                                                                                                                          

 36%|███████████████████████████████████████████████████████████████▎                                                                                                                | 36/100 [00:33<00:49,  1.30it/s]

 37%|█████████████████████████████████████████████████████████████████                                                                                                               | 37/100 [00:34<00:47,  1.33it/s]

 38%|██████████████████████████████████████████████████████████████████▉                                                                                                             | 38/100 [00:35<00:45,  1.35it/s]

 39%|████████████████████████████████████████████████████████████████████▋                                                                                                           | 39/100 [00:35<00:45,  1.35it/s]

 40%|██████████████████████████████████████████████████████████████████████▍                                                            

 72%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                 | 72/100 [01:07<00:27,  1.02it/s]

 73%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                               | 73/100 [01:08<00:26,  1.03it/s]

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                             | 74/100 [01:09<00:25,  1.03it/s]

 75%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                            | 75/100 [01:10<00:24,  1.02it/s]

 76%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

              precision    recall  f1-score   support

           0       0.59      0.72      0.65     13938
           1       0.32      0.26      0.29      6196
           2       0.36      0.26      0.30      3563
           3       0.42      0.39      0.40      3110
           4       0.14      0.05      0.07      1248

    accuracy                           0.49     28055
   macro avg       0.36      0.33      0.34     28055
weighted avg       0.46      0.49      0.47     28055

